<a href="https://colab.research.google.com/github/EdoardoZappia/DeepLearning_Units_2024/blob/main/1_Learning_representations_by_back_propagating_errors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Database

In [1]:
import torch as th
from torch.utils.data import Dataset, DataLoader
from torch.optim import Optimizer
from typing import List, Union, Callable
import matplotlib.pyplot as plt
import numpy

In [2]:
DEVICE_AUTODETECT: bool = True
device: th.device = th.device("cuda" if th.cuda.is_available() and DEVICE_AUTODETECT else "cpu")

In [3]:
class BinaryNumbersDataset(Dataset):
    def __init__(self):
        self.binary_numbers = self.generate_binary_numbers()

    def __len__(self):
        return len(self.binary_numbers)

    def __getitem__(self, idx):
        num = self.binary_numbers[idx][0]  # Ottieni il numero binario come tensore
        label = self.binary_numbers[idx][1]  # Ottieni l'etichetta come tensore
        return num, label

    def is_symmetric(self, num):
        """
        Verifica se un numero binario è simmetrico.
        """
        return int((num == num.flip(0)).all().item())

    def generate_binary_numbers(self):
        """
        Genera tutti i possibili numeri binari a 6 cifre.
        """
        binary_numbers = []
        for i in range(2 ** 6):  # Genera numeri da 0 a 2^6 - 1
            binary_tensor = th.tensor([i >> d & 1 for d in range(5, -1, -1)])  # Converte in un tensore di interi
            is_symmetric = int(self.is_symmetric(binary_tensor))  # Determina se il numero è simmetrico (1) o meno (0)
            binary_numbers.append((binary_tensor, is_symmetric))
        return binary_numbers

# Creazione del dataset
binary_dataset = BinaryNumbersDataset()

batch_size=64

train_loader: DataLoader = DataLoader(
    dataset=binary_dataset, batch_size=batch_size, shuffle=True
)

In [4]:
def seqmlp(input_size: int, hidden_size: int, output_size: int) -> th.nn.Sequential:
    return th.nn.Sequential(
        th.nn.Linear(
            in_features=input_size, out_features=hidden_size, bias=True
        ),  # First linear transformation
        th.nn.Sigmoid(),  # Sigmoid activation
        th.nn.Linear(
            hidden_size, output_size, bias=True
        ),  # Second linear transformation
    )

In [5]:
model: th.nn.Module = seqmlp(6, 2, 1).to(device)

In [6]:
CRITERION: Union[th.nn.Module, Callable[[th.Tensor], th.Tensor]] = (th.nn.MSELoss())

In [19]:
class CustomOptimizer(Optimizer):
    def __init__(self, params, eps=0.1, alpha=0.9):
        defaults = dict(eps=eps, alpha=alpha)
        super(CustomOptimizer, self).__init__(params, defaults)

    def step(self, closure=None):

        for _ in range(1425):
            # Stampare i valori dei pesi prima dell'ottimizzazione
#            for name, param in model.named_parameters():
#                if param.requires_grad:
#                    print("Before optimization -", name, param.data)

            loss = None
            if closure is not None:
                loss = closure()

            for group in self.param_groups:
                eps = group['eps']
                alpha = group['alpha']

                for p in group['params']:
                    if p.grad is None:
                        continue
                    grad = p.grad.data
                    state = self.state[p]

                    # Inizializza delta w a zero se non è stato ancora inizializzato
                    if 'delta_w' not in state:
                        state['delta_w'] = th.zeros_like(p.data)

                    delta_w = state['delta_w']

                    # Aggiorna delta w
                    delta_w = -eps * grad + alpha * delta_w

                    # Aggiorna i pesi
                    p.data.add_(delta_w)

                    # Salva lo stato aggiornato
                    state['delta_w'] = delta_w

            # Stampare i valori dei pesi dopo l'ottimizzazione
#            for name, param in model.named_parameters():
#                if param.requires_grad:
#                    print("After optimization -", name, param.data)
        return loss


def initialize_weights(module):
    if isinstance(module, th.nn.Linear):
        th.nn.init.uniform_(module.weight, a=-0.3, b=0.3)
        th.nn.init.zeros_(module.bias)

model.apply(initialize_weights)

Sequential(
  (0): Linear(in_features=6, out_features=2, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=2, out_features=1, bias=True)
)

In [20]:
optimizer = CustomOptimizer(model.parameters(), eps=0.1, alpha=0.9)

In [21]:
eval_losses: List[float] = []
eval_acc: List[float] = []
test_acc: List[float] = []

model.train()

    # Loop over data
for x, y in train_loader:
    # Sposta ciascun batch di input e di output sul dispositivo specificato
    x = x.float().to(device)
    y = y.float().to(device)
        # Forward pass + loss computation
    yhat = model(x)
    y = y.view(-1, 1, 1)
    loss = CRITERION(yhat, y)

        # Zero-out past gradients
    optimizer.zero_grad()

        # Backward pass
    loss.backward()

        # Update model parameters
    optimizer.step()

    # Log the loss and accuracy on the training set...
num_elem: int = 0
trackingmetric: float = 0
trackingcorrect: int = 0

In [22]:
model.eval()  # Remember to set the model in evaluation mode before evaluating it

    # Since we are just evaluating the model, we don't need to compute gradients
with th.no_grad():
        # ... by looping over training data again
        for x_e, y_e in train_loader:
            x_e, y_e = x_e.float().to(device), y_e.float().to(device)
            modeltarget_e = model(x_e)
            ypred_e = th.argmax(modeltarget_e, dim=1, keepdim=True)
            trackingmetric += CRITERION(modeltarget_e, y_e).item()
            trackingcorrect += ypred_e.eq(y_e.view_as(ypred_e)).sum().item()
            num_elem += x_e.shape[0]
        eval_losses.append(trackingmetric / num_elem)
        eval_acc.append(trackingcorrect / num_elem)

In [23]:
print(f"Final training loss: {eval_losses[-1]}")
print(f"Final training accuracy: {eval_acc[-1]}")

Final training loss: 241.4180908203125
Final training accuracy: 0.875
